__чистим данные__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# import re
# from os import listdir
# from datetime import datetime as dtm
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
import sys
import logging

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

---

In [4]:
raw_data_path = 'data/raw/' # каталог с файлами объявлений
loc_file_path = 'data/location.pkl' # таблица адресов
result_file_path = 'data/data.pkl'

## собираем и чистим данные

In [5]:
from lib.avito import AvitoDataAggRealty

data = AvitoDataAggRealty(data_path=raw_data_path).transform()
data.sample(2)

[INFO    ] 2022-09-06 12:56:47 | AvitoDataAggRealty: 2 raw data files
[INFO    ] 2022-09-06 12:56:48 | AvitoDataAggRealty: 7896 records


,avito_id,title,price,obj_name,adr,description,avito_page,ts,nrooms,floor,...,area,is_studio,is_apartment,is_part,is_auction,is_openspace,is_roof,is_SNT,priceM,is_last_floor
2306,2504954382,"1-к. квартира, 47,2 м², 7/9 эт.",7552000,ЖК «Доброгород»,"улица Токарева, 7 этап, блок-секция 3","Крупнейший в Севастополе семейный микрорайон «Доброгород»! \n\nПобедитель всероссийской премии «Топ ЖК 2021» в номинации «Лучший жилой комплекс». \n\nИпотека с господдержкой, рассрочка от Застройщика. \n\n«Доброгород» по праву считается лучшим жилым кварталом Се",47,2022-09-02 13:04:00,1,7,...,47.20,False,False,False,False,False,False,False,7.55,False
5027,2522885999,"1-к. квартира, 46 м², 3/5 эт.",7653000,NaN,"Севастополь, улица Челнокова, 17 к1","В продаже квартира у моря на улице Челнокова, \n\nполностью готова к Вашему заселению. \n\nДом с закрытой территорией. \n\nЭксклюзивная планировка с большой комнатой и просторной кухней. \n\nКвартира оборудована тёплыми полами водного нагревания. \n\nУстановлена сис",22,2022-09-05 14:02:00,1,3,...,46.00,False,False,False,False,False,False,False,7.65,False


## ищем геопозицию

### обновляем таблицу адресов

In [7]:
from lib.locator import LocationUpdater
from lib.locator import AddressTransformerSev

loc = LocationUpdater(
        address_transformer=AddressTransformerSev(),
    ).transform(
        adr=data['adr'],
        loc=pd.read_pickle(loc_file_path),
        show_pbar=True,
    )

[INFO    ] 2022-09-06 12:57:24 | LocationUpdater: 1672 addresses in location table
[INFO    ] 2022-09-06 12:57:24 | LocationUpdater: 1712 addresses total
[INFO    ] 2022-09-06 12:57:24 | LocationUpdater: 1672 addresses defined
[INFO    ] 2022-09-06 12:57:24 | LocationUpdater: 40 addresses undefined


  0%|          | 0/40 [00:00<?, ?it/s]

[INFO    ] 2022-09-06 12:58:11 | LocationUpdater: 0 addresses found


In [8]:
loc.sample(3)

,adr,latitude,longitude,truncated
1590,"Севастополь, улица Челнокова, 17 к1",44.59,33.43,False
551,"улица Аллы Оношко, 48",44.64,33.56,False
927,"улица Ленина, 21",44.61,33.53,False


In [9]:
assert len(loc)>0

In [11]:
loc.to_pickle(loc_file_path)

## дополняем данные геометкой

In [12]:
data = data.merge(loc[['adr','latitude','longitude',]],on=['adr'],how='left')

In [13]:
# data.sample(3)

In [15]:
print('всего записей:', len( data) )
print('записей без геометки:', len( data[ data['latitude'].isnull() ] ) )

всего записей: 7896
записей без геометки: 415


In [16]:
data.to_pickle(result_file_path)